# 2.2 Negation as Failure

Negation as Failure (NAF) searches for an absence of data - inferring new facts where a pattern is missing.

There are two critical components to understand when using NAF with Datalog rules, **NOT** and **NOT EXISTS**.

## NOT

When used alone, **NOT** checks for a pattern to be false.

## NOT EXISTS

Subtly different from **NOT**, **NOT EXISTS** looks for missing data.

**NOT EXISTS** searches the data store for the described pattern. If the returned result set is empty, it returns **true** and vice versa.

### The Closed World Assumption

RDFox follows the closed world assumption, which assumes statements to be false if they cannot be proven true.

This means that the pattern can either be provably false or not present in the data at all to trigger the inference.

# Example

The following example highlights the different implications of **NOT** vs **NOT EXISTS** in the context of a vehicle sensor as it approaches a traffic light.

In [143]:
naf_data = """
@prefix : <https://rdfox.com/example#> .

:switchA :hasState :on .

:switchB :hasState :off .

:device01 a :Component;
    :dependsOn :switchA .

:device02 a :Component;
    :dependsOn :switchB .

:device03 a :Component;
    :dependsOn :switchA ;
    :dependsOn :switchB .

:device04 a :Component.

"""

In [144]:
naf_rules = """

[?device, :atLeastOnDependencyOn, true] :-
    [?device, a, :Component],
    [?device, :dependsOn, ?switch],
    NOT (
        [?switch, :hasState, :off] 
    ).

[?device, :noDependenciesOff, true] :-
    [?device, a, :Component],
    NOT EXISTS ?switch IN (
        [?device, :dependsOn, ?switch],
        [?switch, :hasState, :off] 
    ).

"""

### Variable scope

Syntactically, **NOT EXISTS** introduces a new, unbound variable whose scope is local to the negation formula. It can share the name of a variable in the wider body without sharing a binding.

In [145]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=naf_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=naf_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Issue select query
with open("../queries/2_2-NAFQuery.rq", "r") as file:
    naf_query = file.read()
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": naf_query})
assert_response_ok(response, "Failed to run ft select query.")
print('\n=== Messages based on objects in images ===')
print(response.text)


=== Messages based on objects in images ===
?device	?atLeastOnDependencyOn	?noDependenciesOff
<https://rdfox.com/example#device01>	true	true
<https://rdfox.com/example#device02>		
<https://rdfox.com/example#device03>	true	
<https://rdfox.com/example#device04>		true



## Incremental retraction

NAF is an incredibly powerful tool and raises an potentially thorny question when combined with the Incremental Reasoning of RDFox.

What happens when new data is added that fills the gap of an absent triple, whose absence was leading to the inference of another fact?

The previously inferred fact will have to be retracted.

Try importing some new data that contains something from the **NOT EXISTS** atom and see what happens.

Here, we'll introduce new dependencies for **device04** - now depending on **both switchA and switchB**.

In [146]:
new_data = """
@prefix : <https://rdfox.com/example#> .

:device04 a :Component;
    :dependsOn :switchA ;
    :dependsOn :switchB .

"""

In [147]:
# Add the new data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=new_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Issue the same select query
with open("../queries/2_2-NAFQuery.rq", "r") as file:
    naf_query = file.read()
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": naf_query})
assert_response_ok(response, "Failed to run ft select query.")
print('\n=== Messages based on objects in images ===')
print(response.text)


=== Messages based on objects in images ===
?device	?atLeastOnDependencyOn	?noDependenciesOff
<https://rdfox.com/example#device01>	true	true
<https://rdfox.com/example#device02>		
<https://rdfox.com/example#device03>	true	
<https://rdfox.com/example#device04>	true	



## Cycles in rules

A rule, or set of rules, cannot be imported if they create cyclic logic, meaning an inferred triples change the conditions that led to their inference.

This is known as a stratification error - the rule set cannot be stratified (logically chained in a way that terminates).

### Where can cycles appear?

It can be easy to write rules that cannot be stratified when using Aggregation and Negation.

This behavior can be difficult to spot so RDFox will prevent you from importing such rules and will tell you where the cycle occurs.

With a simple example, it's easy to see. Take this rule, can you see the problem?

In [103]:
datalog_rule = """
prefix : <https://rdfox.com/example#>

[?x, a, :Class] :-
    [?x, a, ?c] ,
    NOT [?x, a, :Class] .
"""

# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Get response
response = requests.post(rdfox_server + "/datastores/default/content", data=datalog_rule)

# Catch errors
assert_response_ok(response, "Failed to add rule.")

print(response)

Exception: Failed to add rule.
Status received=400
RuleCompilationException: The program is not stratified because these components of the dependency graph contain cycles through negation and/or aggregation:
======== COMPONENT 1 ========
    <https://rdfox.com/example#Class>[?x] :- rdf:type[?x, ?c], NOT <https://rdfox.com/example#Class>[?x] .
========================================================================================================================


## Where is NAF relevant?

Negation as failure is a particularly powerful tool as the absence of data is a common concept in real-world applications.

### On-device

To encode real-world decision-making rules, map dependencies, repair data, etc.

### Publishing

To generate advanced recommendations, offer detailed search, control user access, etc.

### Construction and Manufacturing

To determine complex compatibilities, comply with schematics and regulations, simulate failure conditions, etc.


## Exercise

Complete the rule `nafRules.dlog` in the `rules` folder so that the query below can be used to directly find the **switches** that require maintenance - that is to say, they have not passed their maintenance check.

### Helpful resources

[Negation syntax forms](https://oxfordsemtech.github.io/DocumentationDevBuild/reasoning.html#negation)

In [172]:
naf_sparql = """
SELECT ?switch ?passedMaintenanceCheck
WHERE {
    ?switch :requiresMaintenance true .
    
    OPTIONAL {SELECT ?switch ?passedMaintenanceCheck
        WHERE {
            ?switch :passedMaintenanceCheck ?passedMaintenanceCheck .
        }
    }
} ORDER BY DESC(?passedMaintenanceCheck) ?switch
"""

Here is a representative sample of the data in `nafData.ttl`.

In [156]:
sample_data = """

:device001 a :Component;
    :dependsOn :switchA .

# All devices depend on at least one switch.

:switchA a :Component ;
    :hasState :on ;
    :passedMaintenanceCheck true  .

:switchB a :Component ;
    :hasState :off ;
    :passedMaintenanceCheck false .

:switchC a :Component ;
    :hasState :on .

"""

### Check your work

Run the query below to verify the results.

In [175]:
# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Get and add data
with open("../data/2_2-NAFData.ttl", "r") as file:
    data = file.read()
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get and add rules
with open("../rules/2_2-NAFRules.dlog", "r") as rule_file:
    datalog_rule = rule_file.read()
response = requests.post(rdfox_server + "/datastores/default/content", data=datalog_rule)
assert_response_ok(response, "Failed to add rule.")

# Issue select query
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": naf_sparql})
assert_response_ok(response, "Failed to run select query.")
print('\n=== Switches Requiring Maintenance ===')
print(response.text)


=== Switches Requiring Maintenance ===
?switch	?passedMaintenanceCheck
<http://example.org/switchF>	false
<http://example.org/switchH>	false
<http://example.org/switchK>	false
<http://example.org/switchO>	false
<http://example.org/switchI>	
<http://example.org/switchX>	



### You should see...

=== Switches Requiring Maintenance ===
|?switch|?passedMaintenanceCheck|
|-----------|-------------|
|<http://example.org/switchF>|	false|
|<http://example.org/switchH>|	false|
|<http://example.org/switchK>|	false|
|<http://example.org/switchO>|	false|
|<http://example.org/switchI>|	|
|<http://example.org/switchX>|	|